In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

170500096/170498071 [==============================] - 208s 1us/step


In [3]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))

In [4]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [5]:
optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

### callbacks (early stopping, model checkpoint)

In [7]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]

In [8]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 10s 7ms/step - loss: 4.2130 - accuracy: 0.1660 - val_loss: 2.1554 - val_accuracy: 0.2294
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 2.0640 - accuracy: 0.2488 - val_loss: 2.0648 - val_accuracy: 0.2400
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.9457 - accuracy: 0.2902 - val_loss: 1.9147 - val_accuracy: 0.3012
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.8674 - accuracy: 0.3158 - val_loss: 1.8806 - val_accuracy: 0.3162
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.8027 - accuracy: 0.3425 - val_loss: 1.8651 - val_accuracy: 0.3206
Epoch 6/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.7484 - accuracy: 0.3672 - val_loss: 1.7450 - val_accuracy: 0.3696
Epoch 7/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.7000 - accuracy: 0.3827 - val_loss: 1.7038 - v

In [9]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.5032 - accuracy: 0.4648


[1.5032280683517456, 0.46480000019073486]

### Batch normalization

In [10]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.8405 - accuracy: 0.3415 - val_loss: 1.6922 - val_accuracy: 0.4030
Epoch 2/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.6685 - accuracy: 0.4045 - val_loss: 1.5868 - val_accuracy: 0.4324
Epoch 3/100
1407/1407 [==============================] - 16s 12ms/step - loss: 1.5968 - accuracy: 0.4323 - val_loss: 1.5120 - val_accuracy: 0.4578
Epoch 4/100
1407/1407 [==============================] - 16s 12ms/step - loss: 1.5500 - accuracy: 0.4457 - val_loss: 1.4961 - val_accuracy: 0.4628
Epoch 5/100
1407/1407 [==============================] - 16s 12ms/step - loss: 1.5017 - accuracy: 0.4666 - val_loss: 1.4365 - val_accuracy: 0.4928
Epoch 6/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.4654 - accuracy: 0.4776 - val_loss: 1.4183 - val_accuracy: 0.4880
Epoch 7/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.4328 - accuracy: 0.4906 - val_loss: 1

[1.3231035470962524, 0.5360000133514404]

### Selu

In [11]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.9451 - accuracy: 0.3021 - val_loss: 1.8620 - val_accuracy: 0.3352
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.7210 - accuracy: 0.3877 - val_loss: 1.7397 - val_accuracy: 0.3904
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6180 - accuracy: 0.4287 - val_loss: 1.7237 - val_accuracy: 0.3930
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5502 - accuracy: 0.4527 - val_loss: 1.6405 - val_accuracy: 0.4314
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4881 - accuracy: 0.4741 - val_loss: 1.5681 - val_accuracy: 0.4496
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4380 - accuracy: 0.4972 - val_loss: 1.5565 - val_accuracy: 0.4598
Epoch 7/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.3979 - accuracy: 0.5127 - val_loss: 1.5250 -

[1.465861439704895, 0.5113999843597412]

In [12]:
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - ETA: 0s - loss: 1.4700 - accuracy: 0.50 - 0s 2ms/step - loss: 1.4659 - accuracy: 0.5114


[1.465861439704895, 0.5113999843597412]

### Alpha dropout , MC dropout

In [13]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)


Epoch 1/100
1407/1407 [==============================] - 12s 9ms/step - loss: 1.8984 - accuracy: 0.3247 - val_loss: 1.8221 - val_accuracy: 0.3534
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6678 - accuracy: 0.4130 - val_loss: 1.6902 - val_accuracy: 0.4036
Epoch 3/100
1407/1407 [==============================] - 11s 7ms/step - loss: 1.5795 - accuracy: 0.4476 - val_loss: 1.6495 - val_accuracy: 0.4198
Epoch 4/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.5132 - accuracy: 0.4668 - val_loss: 1.5818 - val_accuracy: 0.4572
Epoch 5/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.4477 - accuracy: 0.4937 - val_loss: 1.5930 - val_accuracy: 0.4670
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4020 - accuracy: 0.5081 - val_loss: 1.5108 - val_accuracy: 0.4882
Epoch 7/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.3606 - accuracy: 0.5275 - val_loss: 1.5762 -

[1.5107836723327637, 0.48820000886917114]

In [14]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [15]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [16]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

0.4876

### 1cycle scheduling

In [20]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(lr=1e-2)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [22]:
K = keras.backend


class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [23]:
n_epochs = 100
onecycle = OneCycleScheduler(len(X_train_scaled) // batch_size * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[onecycle,keras.callbacks.EarlyStopping(patience=20)])

Epoch 1/100
352/352 [==============================] - 3s 9ms/step - loss: 2.0605 - accuracy: 0.2827 - val_loss: 1.7523 - val_accuracy: 0.3842
Epoch 2/100
352/352 [==============================] - 3s 9ms/step - loss: 1.7928 - accuracy: 0.3700 - val_loss: 1.6468 - val_accuracy: 0.4268
Epoch 3/100
352/352 [==============================] - 3s 9ms/step - loss: 1.6657 - accuracy: 0.4127 - val_loss: 1.6190 - val_accuracy: 0.4260
Epoch 4/100
352/352 [==============================] - 3s 9ms/step - loss: 1.5773 - accuracy: 0.4419 - val_loss: 1.5698 - val_accuracy: 0.4530
Epoch 5/100
352/352 [==============================] - 3s 9ms/step - loss: 1.5116 - accuracy: 0.4643 - val_loss: 1.5651 - val_accuracy: 0.4502
Epoch 6/100
352/352 [==============================] - 3s 9ms/step - loss: 1.4562 - accuracy: 0.4810 - val_loss: 1.5435 - val_accuracy: 0.4596
Epoch 7/100
352/352 [==============================] - 3s 9ms/step - loss: 1.4118 - accuracy: 0.4981 - val_loss: 1.5522 - val_accuracy: 0.4638